# Shesha Tutorial: CRISPR Perturbation Analysis

This notebook demonstrates how to use `shesha.bio` to analyze single-cell CRISPR perturbation experiments with AnnData.

**What you'll learn:**
- How to compute perturbation stability (do cells respond consistently?)
- How to compute effect size (how strong is the perturbation?)
- How to interpret the relationship between stability and effect size

**Requirements:**
```bash
pip install shesha-geometry pertpy scanpy anndata
```

### References
* **Norman et al. (2019).** *Science*. "Exploring genetic interaction manifolds constructed from rich single-cell phenotypes." [10.1126/science.aax4438](https://doi.org/10.1126/science.aax4438)

## 1. Setup and Imports

In [ ]:
# Optional: Install dependencies
# !pip install shesha-geometry pertpy scanpy anndata 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr, linregress

# Shesha
from shesha.bio import compute_stability, compute_magnitude

# Single-cell tools
import scanpy as sc
import pertpy as pt

# Configuration
SEED = 320
np.random.seed(SEED)
N_PCA_DIMS = 50

print("Imports successful!")

## 2. Load the Norman 2019 Dataset

We'll use the Norman 2019 CRISPRa dataset, which contains ~111k cells with various gene activations.

In [ ]:
# Load dataset (this may take a minute)
print("Loading Norman 2019 dataset...")
adata = pt.dt.norman_2019()
print(f"Dataset shape: {adata.shape[0]} cells x {adata.shape[1]} genes")

In [ ]:
# Check available metadata
print("Available columns:")
print(adata.obs.columns.tolist())

## 3. Preprocessing

Standard single-cell preprocessing: normalize, log-transform, select highly variable genes, and run PCA.

In [ ]:
# Preprocess (Standard Scanpy Workflow)
sc.pp.calculate_qc_metrics(adata, inplace=True)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
sc.pp.pca(adata, n_comps=50)

# Create a "Proxy" AnnData for Shesha
# We create a new object where .X is the PCA coordinates.
# This allows Shesha to calculate geometric stability on the robust PCA space.
from anndata import AnnData
adata_pca = AnnData(X=adata.obsm['X_pca'], obs=adata.obs)

## 4. Compute Shesha Metrics

Instead of manually slicing arrays, we use the high-level Shesha API. 

We compute metrics on the **PCA embeddings** (`adata_pca`) rather than raw counts to capture robust geometric relationships.

In [ ]:
print("Computing geometric stability...")

# 1. Compute Stability (Precision)
# Measures how consistent the perturbation direction is across cells.
stability_scores = compute_stability(
    adata_pca,
    perturbation_key='perturbation_name',
    control_label='control',
    metric='cosine'
)

# 2. Compute Magnitude (Strength)
# Measures how far the perturbation shifts cells from the control state.
magnitude_scores = compute_magnitude(
    adata_pca,
    perturbation_key='perturbation_name',
    control_label='control',
    metric='euclidean'
)

# 3. Organize results into a DataFrame
df = pd.DataFrame({
    'perturbation': stability_scores.keys(),
    'stability': stability_scores.values(),
    'magnitude': [magnitude_scores[k] for k in stability_scores.keys()]
})

# Add cell counts (critical for filtering noise later)
counts = adata.obs['perturbation_name'].value_counts()
df['n_cells'] = df['perturbation'].map(counts)

print(f"Computed metrics for {len(df)} perturbations.")
df.head()

## 5. Correlation Analysis

A key finding: stability and effect size are strongly correlated. Stronger perturbations produce more consistent cellular responses.

In [ ]:
# Compute correlation
# Note: We use 'magnitude' because that is the column name we created in Section 4
rho, p_value = spearmanr(df['stability'], df['magnitude'])

print(f"Spearman correlation (Stability vs Magnitude):")
print(f"  rho = {rho:.3f}")
print(f"  p   = {p_value:.2e}")

## 6. Visualization
> **Note on Reproduction vs. Paper Figure**
>
> You may notice this plot shows a cleaner separation between discordant groups than the corresponding figure (Figure 34) in the *Geometric Stability: The Missing Axis of Representations*.
>
> This is intentional. The original paper analysis used a permissive filter (`n_cells > 10`) and grouped entire gene families together (e.g., all CEBPs) to demonstrate the metric's robustness to noise and weaker drivers. In this tutorial, we apply a stricter quality filter (`n_cells > 50`) and isolate the strongest driver (`CEBPA`) to clearly illustrate the biological principle of **Discordance**.

In [ ]:
# 1. Filter out noisy low-cell-count perturbations
df_clean = df[df['n_cells'] > 50].copy()

# 2. Calculate Linear Regression (Fit Line)
slope, intercept, r_value, p_value, std_err = linregress(df_clean['magnitude'], df_clean['stability'])

plt.figure(figsize=(10, 7))
sns.set_style("whitegrid")

# Plot "Other" points
sns.scatterplot(
    data=df_clean, 
    x='magnitude', 
    y='stability', 
    color='lightgray', 
    alpha=0.5, 
    label='Other',
    edgecolor=None,
    s=40
)

# Highlight KLF1
klf1 = df_clean[df_clean['perturbation'].str.contains('KLF1')]
plt.scatter(
    klf1['magnitude'], 
    klf1['stability'], 
    color='#1f77b4', 
    label='KLF1 (Lineage Specific)', 
    edgecolor='white', 
    s=80
)

# Highlight CEBPA
cebpa = df_clean[df_clean['perturbation'].str.contains('CEBPA')]
plt.scatter(
    cebpa['magnitude'], 
    cebpa['stability'], 
    color='#d62728', 
    label='CEBPA (Pleiotropic)', 
    edgecolor='white', 
    s=80
)

# Add the Fit Line
x_vals = np.array([df_clean['magnitude'].min(), df_clean['magnitude'].max()])
y_vals = slope * x_vals + intercept
plt.plot(x_vals, y_vals, 'k--', alpha=0.7, label=f'Linear fit (R={r_value:.2f})')

# Labels
plt.xlabel('Effect Magnitude (Euclidean Distance)', fontsize=12)
plt.ylabel('Geometric Stability (Cosine Consistency)', fontsize=12)
plt.title('Discordance Plot: Specificity vs. Pleiotropy', fontsize=14)
plt.legend(frameon=True)
plt.show()

## 7. Top and Bottom Perturbations

Let's look at which perturbations have the highest and lowest stability.

In [ ]:
# Sort by stability
df_sorted = df.sort_values('stability', ascending=False)

print("TOP 10 - Most Stable Perturbations:")
print("=" * 60)
print(df_sorted.head(10).to_string(index=False))

print("\n\nBOTTOM 10 - Least Stable Perturbations:")
print("=" * 60)
print(df_sorted.tail(10).to_string(index=False))

## 8. Interpretation (Norman 2019)

### What does high stability mean?
* **Robust Biological Signal:** Cells respond to the perturbation in a **consistent direction** across the population.
* **High Phenotypic Penetrance:** The perturbation is strong enough to overcome stochastic gene expression noise, driving all cells into a coherent state.
* **Likely "On-Target":** Indicates a specific, clean regulatory effect.
    * *Example:* **KLF1** shows high stability because it drives a precise, lineage-specific gene program (erythroid differentiation).

### What does low stability mean?
* **Heterogeneous Response:** Cells respond **heterogeneously** to the perturbation, resulting in low geometric consistency.
* **Context Dependence:** The effect depends on the initial cell state (e.g., cell cycle phase or chromatin accessibility) rather than being universal.
    * *Example:* **CEBPA** shows lower stability relative to its magnitude. As a "pioneer factor," its effect varies depending on the chromatin environment of each individual cell.
* **Technical Noise:** Can indicate mosaicism (some cells unperturbed) or low guide efficiency.

### The Stability-Magnitude Correlation
In the Norman et al. dataset, we observe a strong positive correlation (**$\rho \approx 0.95$**). This suggests that **stronger perturbations are generally more consistent**.

This aligns with the **Signal-to-Noise Ratio (SNR)** limitations of single-cell screens described in foundational studies (e.g., Adamson et al., 2016). We can interpret this relationship as follows:

> **The Signal-to-Noise Principle:**
>
> * **High Magnitude (Signal):** Acts as a dominant force that overrides cellular noise, pushing all cells uniformly down a trajectory.
> * **Low Magnitude (Noise):** Weak perturbations often fail to rise above the stochastic "buzz" of background gene expression, leading to unstable or random directionality.
---

### References
* **Adamson et al. (2016).** *Cell*. "A Multiplexed Single-Cell CRISPR Screening
Platform Enables Systematic Dissection of the
Unfolded Protein Response." [10.1016/j.cell.2016.11.048](http://dx.doi.org/10.1016/j.cell.2016.11.048) (High-magnitude perturbations are required to distinguish signal from biological noise).
* **Dixit et al. (2016).** *Cell*. "Perturb-Seq: Dissecting Molecular Circuits with Scalable Single-Cell RNA Profiling of Pooled Genetic Screens" [10.1016/j.cell.2016.11.038](https://doi.org/10.1016/j.cell.2016.11.038) (Discusses how mosaicism and cell state affect single-cell CRISPR readouts).

## 9. Quick Reference

```python
from shesha.bio import compute_stability, compute_magnitude
```

**Step 1: Create a PCA Proxy (Recommended)**
Shesha calculates metrics on `.X`. To measure geometric stability on robust PCA coordinates (50 dims) rather than noisy gene counts, create a lightweight proxy:
```python
# Create a view where .X is the PCA matrix
from anndata import AnnData
adata_pca = AnnData(X=adata.obsm['X_pca'], obs=adata.obs)
```

**Step 2: Compute Metrics** Use the high-level API to compute scores for all perturbations at once

```python
# Compute Stability (Consistency)
stability = compute_stability(
    adata_pca, 
    perturbation_key="perturbation_name", 
    control_label="control",
    metric="cosine"
)

# Compute Magnitude (Strength)
magnitude = compute_magnitude(
    adata_pca, 
    perturbation_key="perturbation_name", 
    control_label="control", 
    metric="euclidean"
)
```

**Typical values:**
- Stability > 0.5: Strong, consistent perturbation (e.g., Lineage Drivers like KLF1).
- Stability 0.2-0.5: Moderate perturbation.
- Stability < 0.2: Weak or heterogeneous perturbation (e.g., Pleiotropic Factors like CEBPA).
- High Magnitude + High Stability: Ideal target for specific therapeutic intervention.
- High Magnitude + Low Stability: "Promiscuous" regulator; likely to have broad, variable side effects.
